In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
# !pip install matplotlib numpy pandas tqdm nltk

# for separating ingredients vs non-ingredients
# NOTE: if using Windows to run this, need to download GNU Wget
# !wget -c https://raw.githubusercontent.com/williamLyh/RecipeWithPlans/main/ingredient_set.json -O ingredient_set.json

In [3]:
import os
import math
import re
import string
import numpy as np
import pandas as pd
import random
import json
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR, MultiStepLR, CosineAnnealingWarmRestarts
import nltk
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from nltk.translate import meteor

from data import *
from encoder_decoder import *
from train import *
from eval import *
from utils import *

# required for bleu
# nltk.download("wordnet")

/home/student/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

In [4]:
SEED = 31989101
HIDDEN_SIZE = 256
MAX_INGR_LEN = 150 # fixed from assignment
MAX_RECIPE_LEN = 600
DROPOUT = 0.1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## ensuring reproducibility
def reset_rng():
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

reset_rng()

# to easily read ingredients and instructions
pd.set_option('display.max_colwidth', 2000)

print(f"Using device: {DEVICE}")

Using device: cuda


In [5]:
data_root = "./Cooking_Dataset"
add_intermediate_tag=False

train_df_orig = pd.read_csv(os.path.join(data_root, "train.csv"), usecols=['Ingredients', 'Recipe'])
dev_df_orig = pd.read_csv(os.path.join(data_root, "dev.csv"), usecols=['Ingredients', 'Recipe'])
test_df_orig = pd.read_csv(os.path.join(data_root, "test.csv"), usecols=['Ingredients', 'Recipe'])

In [6]:
train_df = preprocess_data(train_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)

Number of data samples before preprocessing: 101340
Number of data samples after preprocessing: 100637 (99.306%)


In [7]:
dev_df = preprocess_data(dev_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)

Number of data samples before preprocessing: 797
Number of data samples after preprocessing: 793 (99.498%)


In [8]:
test_df = preprocess_data(test_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)

Number of data samples before preprocessing: 778
Number of data samples after preprocessing: 774 (99.486%)


In [9]:
vocab = Vocabulary(add_intermediate_tag=add_intermediate_tag)
vocab.populate(train_df)
vocab.n_unique_words

  2%|▏         | 1555/100637 [00:00<00:06, 15547.09it/s]

100%|██████████| 100637/100637 [00:06<00:00, 15152.13it/s]


44315

In [10]:
train_ds = RecipeDataset(train_df, vocab)
# subset_train_ds = RecipeDataset(train_df[:250], vocab) # ! REMOVE LATER
dev_ds_val_loss = RecipeDataset(dev_df, vocab, train=True) # used for getting validation loss
dev_ds_val_met = RecipeDataset(dev_df, vocab, train=False) # used for getting validation BLEU, and other metrics
test_ds = RecipeDataset(test_df, vocab, train=False)

## Encoder-Decoder (Base)

In [11]:
embedding_size=300
encoder = EncoderRNN(vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, padding_value=vocab.word2index(PAD_WORD)).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
decoder = DecoderRNN(embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1).to(DEVICE)

In [14]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_optimizer = optim.Adam(encoder.parameters(), lr=initial_lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=initial_lr)
enc_scheduler = CosineAnnealingLR(encoder_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_scheduler = CosineAnnealingLR(decoder_optimizer, T_max=n_epochs, eta_min=min_lr)
# enc_scheduler = MultiStepLR(encoder_optimizer, milestones=[15], gamma=0.1)
# dec_scheduler = MultiStepLR(decoder_optimizer, milestones=[15], gamma=0.1)
identifier="adam_without_intermediate_tags_wd0_lr1e-3"
epoch_losses, val_epoch_losses, log = train(encoder, decoder, encoder_optimizer, decoder_optimizer, train_ds, 
                     n_epochs=n_epochs, vocab=vocab, decoder_mode="basic", batch_size=batch_size, 
                     enc_lr_scheduler=enc_scheduler, dec_lr_scheduler=dec_scheduler, 
                     dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
                     verbose_iter_interval=10)

save_log(identifier, log, encoder_optimizer, decoder_optimizer, enc_scheduler, dec_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 10/787) Average loss so far: 10.210
(Epoch 0, iter 20/787) Average loss so far: 6.945
(Epoch 0, iter 30/787) Average loss so far: 6.156
(Epoch 0, iter 40/787) Average loss so far: 6.180
(Epoch 0, iter 50/787) Average loss so far: 6.071
(Epoch 0, iter 60/787) Average loss so far: 6.040
(Epoch 0, iter 70/787) Average loss so far: 6.024
(Epoch 0, iter 80/787) Average loss so far: 5.991
(Epoch 0, iter 90/787) Average loss so far: 5.968
(Epoch 0, iter 100/787) Average loss so far: 5.922
(Epoch 0, iter 110/787) Average loss so far: 5.875
(Epoch 0, iter 120/787) Average loss so far: 5.845
(Epoch 0, iter 130/787) Average loss so far: 5.806
(Epoch 0, iter 140/787) Average loss so far: 5.735
(Epoch 0, iter 150/787) Average loss so far: 5.680
(Epoch 0, iter 160/787) Average loss so far: 5.623
(Epoch 0, iter 170/787) Average loss so far: 5.582
(Epoch 0, iter 180/787) Average loss so far: 5.544
(Epoch 0, iter 1

7it [00:03,  2.05it/s]


validation loss: 4.239502702440534


 14%|█▍        | 28/199 [00:15<01:36,  1.77it/s]


KeyboardInterrupt: 

In [ ]:
save_model(encoder, decoder, "adam_without_intermediate_tags_wd0_lr1e-3_last")

## Encoder-Decoder (Attention)

In [11]:
reset_rng()

In [12]:
embedding_size=300
encoder_attn = EncoderRNN(vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, padding_value=vocab.word2index(PAD_WORD)).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
# decoder = DecoderRNN(embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-2).to(DEVICE)
decoder_attn = AttnDecoderRNN(embedding_size, hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1, padding_val=vocab.word2index(PAD_WORD), 
                              dropout=DROPOUT).to(DEVICE)

In [13]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_attn_optimizer = optim.Adam(encoder_attn.parameters(), lr=initial_lr)
decoder_attn_optimizer = optim.Adam(decoder_attn.parameters(), lr=initial_lr)
# enc_attn_scheduler = CosineAnnealingLR(encoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
# dec_attn_scheduler = CosineAnnealingLR(decoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
enc_attn_scheduler = CosineAnnealingLR(encoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_attn_scheduler = CosineAnnealingLR(decoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
identifier="attn_adam_without_intermediate_tags_wd0_lr1e-3"

attn_epoch_losses, attn_val_epoch_losses, attn_log = train(
    encoder_attn, decoder_attn, encoder_attn_optimizer, decoder_attn_optimizer, train_ds, 
    n_epochs=n_epochs, vocab=vocab, decoder_mode="attention", batch_size=batch_size, 
    enc_lr_scheduler=enc_attn_scheduler, dec_lr_scheduler=dec_attn_scheduler, 
    dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
    verbose_iter_interval=50)

save_log(identifier, attn_log, encoder_attn_optimizer, decoder_attn_optimizer, 
         enc_attn_scheduler, dec_attn_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 50/787) Average loss so far: 6.972
(Epoch 0, iter 100/787) Average loss so far: 6.023
(Epoch 0, iter 150/787) Average loss so far: 5.968
(Epoch 0, iter 200/787) Average loss so far: 5.927
(Epoch 0, iter 250/787) Average loss so far: 5.877
(Epoch 0, iter 300/787) Average loss so far: 5.818
(Epoch 0, iter 350/787) Average loss so far: 5.785
(Epoch 0, iter 400/787) Average loss so far: 5.725
(Epoch 0, iter 450/787) Average loss so far: 5.683
(Epoch 0, iter 500/787) Average loss so far: 5.621
(Epoch 0, iter 550/787) Average loss so far: 5.551
(Epoch 0, iter 600/787) Average loss so far: 5.440
(Epoch 0, iter 650/787) Average loss so far: 5.360
(Epoch 0, iter 700/787) Average loss so far: 5.215
(Epoch 0, iter 750/787) Average loss so far: 5.093
Average epoch loss: 5.701
This epoch took 8.194433637460072 mins. Time remaining: 3.0 hrs 57.0 mins.


7it [00:01,  5.45it/s]


validation loss: 4.93167952128819


100%|██████████| 793/793 [00:08<00:00, 95.55it/s] 


BLEU score: 0.002800126985071343, METEOR score: 0.08576382563509898
Starting epoch 2/30, enc lr scheduler: [0.0009972883382072953], dec lr scheduler: [0.0009972883382072953]
(Epoch 1, iter 50/787) Average loss so far: 4.863
(Epoch 1, iter 100/787) Average loss so far: 4.706
(Epoch 1, iter 150/787) Average loss so far: 4.641
(Epoch 1, iter 200/787) Average loss so far: 4.523
(Epoch 1, iter 250/787) Average loss so far: 4.437
(Epoch 1, iter 300/787) Average loss so far: 4.366
(Epoch 1, iter 350/787) Average loss so far: 4.295
(Epoch 1, iter 400/787) Average loss so far: 4.248
(Epoch 1, iter 450/787) Average loss so far: 4.186
(Epoch 1, iter 500/787) Average loss so far: 4.133
(Epoch 1, iter 550/787) Average loss so far: 4.082
(Epoch 1, iter 600/787) Average loss so far: 4.041
(Epoch 1, iter 650/787) Average loss so far: 4.004
(Epoch 1, iter 700/787) Average loss so far: 3.956
(Epoch 1, iter 750/787) Average loss so far: 3.932
Average epoch loss: 4.277
This epoch took 8.073306282361349 mi

...

BLEU score: 0.05513984026373013, METEOR score: 0.2094436932678814
Starting epoch 30/30, enc lr scheduler: [1.2711661792704668e-05], dec lr scheduler: [1.2711661792704668e-05]
(Epoch 29, iter 50/787) Average loss so far: 2.732
(Epoch 29, iter 100/787) Average loss so far: 2.722
(Epoch 29, iter 150/787) Average loss so far: 2.715
(Epoch 29, iter 200/787) Average loss so far: 2.739
(Epoch 29, iter 250/787) Average loss so far: 2.727
(Epoch 29, iter 300/787) Average loss so far: 2.729
(Epoch 29, iter 350/787) Average loss so far: 2.730
(Epoch 29, iter 400/787) Average loss so far: 2.714
(Epoch 29, iter 450/787) Average loss so far: 2.725
(Epoch 29, iter 500/787) Average loss so far: 2.711
(Epoch 29, iter 550/787) Average loss so far: 2.733
(Epoch 29, iter 600/787) Average loss so far: 2.737
(Epoch 29, iter 650/787) Average loss so far: 2.723
(Epoch 29, iter 700/787) Average loss so far: 2.724
(Epoch 29, iter 750/787) Average loss so far: 2.724
Average epoch loss: 2.725
This epoch took 7.99

7it [00:01,  5.51it/s]


validation loss: 3.1326404299054826


100%|██████████| 793/793 [00:01<00:00, 595.70it/s]

BLEU score: 0.055078340546709854, METEOR score: 0.20826842611097787


In [14]:
save_model(encoder_attn, decoder_attn, f"{identifier}_last")

## Encoder-Decoder (Extension: pretrained embeddings)

In [14]:
reset_rng()

In [17]:
#! TODO: PUT IN CODE TO WGET AND EXTRACT FROM ZIP

In [15]:
pretrained_embedding_dict = create_pretrained_embedding_dict("./glove.840B.300d.txt")

In [16]:
embedding_size=300
encoder_pretrained_embed = EncoderRNN(
    input_size=vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, 
    padding_value=vocab.word2index(PAD_WORD), pretrained_embedding_dict=pretrained_embedding_dict, 
    vocab=vocab).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
decoder_pretrained_embed = AttnDecoderRNN(
    embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1,
    padding_val=vocab.word2index(PAD_WORD), dropout=DROPOUT, global_max_ing_len=MAX_INGR_LEN,
    pretrained_embedding_dict=pretrained_embedding_dict, vocab=vocab).to(DEVICE)

100%|██████████| 44315/44315 [00:00<00:00, 788459.19it/s]


29462/44315 (0.665%) words have pretrained embeddings


100%|██████████| 44314/44314 [00:00<00:00, 753367.82it/s]

29462/44314 (0.665%) words have pretrained embeddings


In [18]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_pretrained_embed_optimizer = optim.Adam(encoder_pretrained_embed.parameters(), lr=initial_lr)
decoder_pretrained_embed_optimizer = optim.Adam(decoder_pretrained_embed.parameters(), lr=initial_lr)
# enc_scheduler = CosineAnnealingLR(encoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
# dec_scheduler = CosineAnnealingLR(decoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
enc_pretrained_embed_scheduler = CosineAnnealingLR(encoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_pretrained_embed_scheduler = CosineAnnealingLR(decoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
identifier="pretrained_emb_attn_adam_without_intermediate_tags_wd0_lr1e-3"

pre_epoch_losses, pre_val_epoch_losses, pre_log = train(
    encoder_pretrained_embed, decoder_pretrained_embed, encoder_pretrained_embed_optimizer, decoder_pretrained_embed_optimizer, 
    train_ds, n_epochs=n_epochs, vocab=vocab, decoder_mode="attention", batch_size=batch_size, 
    enc_lr_scheduler=enc_pretrained_embed_scheduler, dec_lr_scheduler=dec_pretrained_embed_scheduler, 
    dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
    verbose_iter_interval=50)

save_log(identifier, pre_log, encoder_pretrained_embed_optimizer, decoder_pretrained_embed_optimizer, 
         enc_pretrained_embed_scheduler, dec_pretrained_embed_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 50/787) Average loss so far: 6.968
(Epoch 0, iter 100/787) Average loss so far: 6.040
(Epoch 0, iter 150/787) Average loss so far: 5.995
(Epoch 0, iter 200/787) Average loss so far: 5.955
(Epoch 0, iter 250/787) Average loss so far: 5.910
(Epoch 0, iter 300/787) Average loss so far: 5.859
(Epoch 0, iter 350/787) Average loss so far: 5.839
(Epoch 0, iter 400/787) Average loss so far: 5.759
(Epoch 0, iter 450/787) Average loss so far: 5.627
(Epoch 0, iter 500/787) Average loss so far: 5.404
(Epoch 0, iter 550/787) Average loss so far: 5.209
(Epoch 0, iter 600/787) Average loss so far: 5.001
(Epoch 0, iter 650/787) Average loss so far: 4.851
(Epoch 0, iter 700/787) Average loss so far: 4.692
(Epoch 0, iter 750/787) Average loss so far: 4.582
Average epoch loss: 5.528
This epoch took 8.08877116839091 mins. Time remaining: 3.0 hrs 54.0 mins.


7it [00:01,  5.60it/s]


validation loss: 4.4685172353472025


100%|██████████| 793/793 [00:09<00:00, 87.13it/s] 


BLEU score: 0.0031624657072353694, METEOR score: 0.0796926100408187
Starting epoch 2/30, enc lr scheduler: [0.0009972883382072953], dec lr scheduler: [0.0009972883382072953]
(Epoch 1, iter 50/787) Average loss so far: 4.410
(Epoch 1, iter 100/787) Average loss so far: 4.295
(Epoch 1, iter 150/787) Average loss so far: 4.269
(Epoch 1, iter 200/787) Average loss so far: 4.187
(Epoch 1, iter 250/787) Average loss so far: 4.128
(Epoch 1, iter 300/787) Average loss so far: 4.080
(Epoch 1, iter 350/787) Average loss so far: 4.027
(Epoch 1, iter 400/787) Average loss so far: 3.997
(Epoch 1, iter 450/787) Average loss so far: 3.949
(Epoch 1, iter 500/787) Average loss so far: 3.910
(Epoch 1, iter 550/787) Average loss so far: 3.871
(Epoch 1, iter 600/787) Average loss so far: 3.844
(Epoch 1, iter 650/787) Average loss so far: 3.814
(Epoch 1, iter 700/787) Average loss so far: 3.778
(Epoch 1, iter 750/787) Average loss so far: 3.759
Average epoch loss: 4.009
This epoch took 8.075345408916473 mi

...

BLEU score: 0.054989968499987295, METEOR score: 0.21584912823420402
Starting epoch 30/30, enc lr scheduler: [1.2711661792704668e-05], dec lr scheduler: [1.2711661792704668e-05]
(Epoch 29, iter 50/787) Average loss so far: 2.667
(Epoch 29, iter 100/787) Average loss so far: 2.655
(Epoch 29, iter 150/787) Average loss so far: 2.648
(Epoch 29, iter 200/787) Average loss so far: 2.672
(Epoch 29, iter 250/787) Average loss so far: 2.658
(Epoch 29, iter 300/787) Average loss so far: 2.662
(Epoch 29, iter 350/787) Average loss so far: 2.664
(Epoch 29, iter 400/787) Average loss so far: 2.649
(Epoch 29, iter 450/787) Average loss so far: 2.657
(Epoch 29, iter 500/787) Average loss so far: 2.646
(Epoch 29, iter 550/787) Average loss so far: 2.667
(Epoch 29, iter 600/787) Average loss so far: 2.670
(Epoch 29, iter 650/787) Average loss so far: 2.655
(Epoch 29, iter 700/787) Average loss so far: 2.656
(Epoch 29, iter 750/787) Average loss so far: 2.661
Average epoch loss: 2.658
This epoch took 7.

7it [00:01,  6.06it/s]


validation loss: 3.0425445352281844


100%|██████████| 793/793 [00:01<00:00, 619.58it/s]


BLEU score: 0.057802547829115924, METEOR score: 0.21431412239195574


## Extension 4: Multi-Layer Encoder-Decoder + Neurologic Decoding

In [11]:
reset_rng()

In [12]:
embedding_size=300
num_layers=3
encoder_multilayer_attn = EncoderRNN(vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, 
                          padding_value=vocab.word2index(PAD_WORD), num_lstm_layers=num_layers).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
# decoder = DecoderRNN(embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-2).to(DEVICE)
decoder_multilayer_attn = AttnDecoderRNN(embedding_size, hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1, 
                              padding_val=vocab.word2index(PAD_WORD), dropout=DROPOUT, num_lstm_layers=num_layers).to(DEVICE)

In [13]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_multilayer_attn_optimizer = optim.Adam(encoder_multilayer_attn.parameters(), lr=initial_lr)
decoder_multilayer_attn_optimizer = optim.Adam(decoder_multilayer_attn.parameters(), lr=initial_lr)
enc_multilayer_attn_scheduler = CosineAnnealingLR(encoder_multilayer_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_multilayer_attn_scheduler = CosineAnnealingLR(decoder_multilayer_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
identifier="multilayer_attn_adam_without_intermediate_tags_wd0_lr1e-3"

attn_epoch_losses, attn_val_epoch_losses, attn_log = train(
    encoder_multilayer_attn, decoder_multilayer_attn, encoder_multilayer_attn_optimizer, decoder_multilayer_attn_optimizer, train_ds, 
    n_epochs=n_epochs, vocab=vocab, decoder_mode="attention", batch_size=batch_size, 
    enc_lr_scheduler=enc_multilayer_attn_scheduler, dec_lr_scheduler=dec_multilayer_attn_scheduler, 
    dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
    verbose_iter_interval=10)

save_log(identifier, attn_log, encoder_multilayer_attn_optimizer, decoder_multilayer_attn_optimizer, 
         enc_multilayer_attn_scheduler, dec_multilayer_attn_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 10/787) Average loss so far: 9.766
(Epoch 0, iter 20/787) Average loss so far: 6.787
(Epoch 0, iter 30/787) Average loss so far: 6.162
(Epoch 0, iter 40/787) Average loss so far: 6.129
(Epoch 0, iter 50/787) Average loss so far: 6.097
(Epoch 0, iter 60/787) Average loss so far: 6.059
(Epoch 0, iter 70/787) Average loss so far: 6.053
(Epoch 0, iter 80/787) Average loss so far: 6.079
(Epoch 0, iter 90/787) Average loss so far: 6.049
(Epoch 0, iter 100/787) Average loss so far: 6.047
(Epoch 0, iter 110/787) Average loss so far: 6.048
(Epoch 0, iter 120/787) Average loss so far: 6.034
(Epoch 0, iter 130/787) Average loss so far: 6.023
(Epoch 0, iter 140/787) Average loss so far: 6.032
(Epoch 0, iter 150/787) Average loss so far: 6.036
(Epoch 0, iter 160/787) Average loss so far: 6.031
(Epoch 0, iter 170/787) Average loss so far: 6.026
(Epoch 0, iter 180/787) Average loss so far: 6.046
(Epoch 0, iter 19

KeyboardInterrupt: 

---

## Evaluation

### Without attention

In [12]:
load_model(encoder, decoder, "adam_without_intermediate_tags_with_val_wd0_lr1e-3_ep_24")

In [13]:
all_decoder_outs, all_gt_recipes = eval(encoder, decoder, test_ds, vocab, batch_size=128, decoder_mode="basic",
                                        max_recipe_len=MAX_RECIPE_LEN)

100%|██████████| 7/7 [00:02<00:00,  3.07it/s]


In [14]:
calc_bleu(all_gt_recipes, all_decoder_outs)

0.044537752361233085

In [15]:
calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

100%|██████████| 774/774 [00:01<00:00, 435.87it/s]


0.20157579551309057

### With attention

In [15]:
load_model(encoder_attn, decoder_attn, "attn_adam_without_intermediate_tags_wd0_lr1e-3_ep_26")

In [16]:
all_decoder_outs, all_gt_recipes = eval(encoder_attn, decoder_attn, test_ds, vocab, batch_size=128, decoder_mode="attention",
                                        max_recipe_len=MAX_RECIPE_LEN)

100%|██████████| 7/7 [00:02<00:00,  2.40it/s]


In [18]:
all_gt_recipes[0]

['<RECIPE_START>',
 'cream',
 'butter',
 'and',
 'sugar',
 'add',
 'egg',
 'yolks',
 ',',
 'one',
 'at',
 'a',
 'time',
 'and',
 'beat',
 'until',
 'mixture',
 'is',
 'very',
 'light',
 'and',
 'lemon',
 'colored',
 'add',
 'vanilla',
 'and',
 'milk',
 'in',
 'another',
 'bowl',
 'beat',
 'egg',
 'whites',
 'until',
 'foamy',
 'fold',
 'into',
 'creamed',
 'mixture',
 'fold',
 'in',
 '11/2',
 'cups',
 'of',
 'the',
 'coconut',
 'pour',
 'into',
 'baked',
 'pie',
 'shell',
 'and',
 'sprinkle',
 'with',
 'nutmeg',
 'top',
 'with',
 'remaining',
 'grated',
 'coconut',
 'bake',
 'in',
 'preheated',
 '350',
 'oven',
 'for',
 'about',
 '35',
 'minutes',
 'or',
 'until',
 'filling',
 'is',
 'just',
 'set',
 'serve',
 'warm',
 'or',
 'cold',
 'courtesy',
 'of',
 'dale',
 '&',
 'gail',
 'shipp',
 ',',
 'columbia',
 'md',
 '<RECIPE_END>']

In [19]:
all_decoder_outs[0]

['<RECIPE_START>',
 'in',
 'a',
 'medium',
 'saucepan',
 ',',
 'combine',
 'the',
 'sugar',
 'and',
 'cocoa',
 ',',
 'and',
 'cook',
 'over',
 'medium',
 'heat',
 ',',
 'stirring',
 'constantly',
 ',',
 'until',
 'the',
 'mixture',
 'is',
 'smooth',
 'and',
 'the',
 'sugar',
 'is',
 'dissolved',
 ',',
 'about',
 '5',
 'minutes',
 'remove',
 'from',
 'the',
 'heat',
 'and',
 'stir',
 'in',
 'the',
 'milk',
 ',',
 'vanilla',
 'and',
 'salt',
 'and',
 'mix',
 'well',
 'pour',
 'into',
 'a',
 '9',
 '-',
 'inch',
 'springform',
 'pan',
 'bake',
 'in',
 'a',
 'preheated',
 '350',
 'degree',
 'f',
 'oven',
 'for',
 '45',
 'minutes',
 'or',
 'until',
 'the',
 'custard',
 'is',
 'set',
 'cool',
 'completely',
 'on',
 'a',
 'wire',
 'rack',
 'and',
 'refrigerate',
 'for',
 'at',
 'least',
 '2',
 'hours',
 'before',
 'serving',
 '<RECIPE_END>']

In [17]:
calc_bleu(all_gt_recipes, all_decoder_outs)

0.04949963728618708

In [20]:
calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

100%|██████████| 774/774 [00:01<00:00, 636.00it/s]


0.20855536635420194

---

In [19]:
all_decoder_outs, all_gt_recipes = eval(encoder_pretrained_embed, decoder_pretrained_embed, test_ds, vocab, batch_size=128, decoder_mode="attention",
                                        max_recipe_len=MAX_RECIPE_LEN)

100%|██████████| 7/7 [00:03<00:00,  2.11it/s]


In [20]:
calc_bleu(all_gt_recipes, all_decoder_outs)

0.05519142942964554

In [21]:
calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

100%|██████████| 774/774 [00:01<00:00, 586.11it/s]


0.2100281888663272

---

## Metric Sample

In [ ]:
all_ings = get_all_ingredients("./ingredient_set.json")
all_ings_regex = get_ingredients_regex(all_ings)
metric_sample_ings, metric_sample_gold_recipe, metric_sample_generated_recipe = \
    load_metric_sample("./metric_sample.txt")

In [ ]:
prop_inp_ings, n_extra_ings = get_prop_input_num_extra_ingredients(
    metric_sample_ings, metric_sample_generated_recipe, all_ings_regex, verbose=True,
    metric_sample=True)
print(f"\nproportion of input ingredients: {prop_inp_ings}\nnumber of extra ingredients: {n_extra_ings}")

In [ ]:
bleu_score = calc_bleu([metric_sample_gold_recipe], [metric_sample_generated_recipe], split_gen=True)
meteor_score = calc_meteor([metric_sample_gold_recipe], [metric_sample_generated_recipe], split_gen=True)
print(f"BLEU score: {bleu_score}, METEOR score: {meteor_score}")